# ![GA Logo](https://camo.githubusercontent.com/6ce15b81c1f06d716d753a61f5db22375fa684da/68747470733a2f2f67612d646173682e73332e616d617a6f6e6177732e636f6d2f70726f64756374696f6e2f6173736574732f6c6f676f2d39663838616536633963333837313639306533333238306663663535376633332e706e67) Naive Bayes

We've looked at the Naive Bayes classifier from a probability point of view. Now let's apply code to it to a natural language processing problem.

### Before we begin... what is natural language processing?

- If I'm explaining this to my non-technical peers, natural language processing is just a way for us to get computers to understand written language the way you and I do.

- If I'm explaining this to someone with a more technical background, natural language processing is a set of tools that represent words as numbers. This is commonly done by feature engineering (i.e. turning words into columns in your dataframe), but more complicated methods exist.

You'll often see natural language processing abbreviated as **NLP**.

You and I will use these social media posts in a Naive Bayes classification model to predict whether a post comes from Twitter or Facebook.

#### First: some data cleaning.

In [16]:
import pandas as pd

In [17]:
# Read in the .csv file.

df = pd.read_csv("./unprocessed_tweets.csv", encoding='latin-1')

In [18]:
# 'Hello r u interested'
# P *hello | spam)
# P(r | Hello, Spam) --> P(r | spam)
# P(u | Hello r, spam) -- > P(u | spam)
# P (interested | hello r u ) --> P (interested | spam)

# Naive bayes is compatible with bags of words model 
# when we compute the probability, we just multiply everything together 
# P(hello | spam)^2 * P(r | spam) *P(u |spam) *P(interested | spam)

#because we want to compare with P(not spam) --> we cancel away the denominator P(helo r u interested)
# P(spam | "hello r u interested")
# first thing we do --> P(hello r u interested | spam) * P (spam) / P(hello r u interested)
# Second, cancel P(hello r u interested)  * P(spam)

# some bayesian terminology:
# prior (or prior probability) --> P(spam) vs P(not spam)
# you can infer that the probability from data 

# spam.value_counts(normalize = True)

# lets say youre using smoe data set taht someone created mechanical turk? Amazon 
# 50-50 of spam vs not spam for my DS workflow 
# spam.value_counts(normalize = True)


# in log space we add 
# but in normal space we multiply 

NameError: name 'spam' is not defined

In [19]:
# Check out the columns.



In [20]:
# See the first five rows.

df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,audience,audience:confidence,bias,bias:confidence,message,...,orig__golden,audience_gold,bias_gold,bioid,embed,id,label,message_gold,source,text
0,766192484,False,finalized,1,8/4/15 21:17,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,R000596,"<blockquote class=""twitter-tweet"" width=""450"">...",3.83249E+17,From: Trey Radel (Representative from Florida),NaN,twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,False,finalized,1,8/4/15 21:20,national,1.0,partisan,1.0,attack,...,NaN,NaN,NaN,M000355,"<blockquote class=""twitter-tweet"" width=""450"">...",3.11208E+17,From: Mitch McConnell (Senator from Kentucky),NaN,twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,False,finalized,1,8/4/15 21:14,national,1.0,neutral,1.0,support,...,NaN,NaN,NaN,S001180,"<blockquote class=""twitter-tweet"" width=""450"">...",3.39069E+17,From: Kurt Schrader (Representative from Oregon),NaN,twitter,Please join me today in remembering our fallen...
3,766192487,False,finalized,1,8/4/15 21:08,national,1.0,neutral,1.0,policy,...,NaN,NaN,NaN,C000880,"<blockquote class=""twitter-tweet"" width=""450"">...",2.98528E+17,From: Michael Crapo (Senator from Idaho),NaN,twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,False,finalized,1,8/4/15 21:26,national,1.0,partisan,1.0,policy,...,NaN,NaN,NaN,U000038,"<blockquote class=""twitter-tweet"" width=""450"">...",4.07643E+17,From: Mark Udall (Senator from Colorado),NaN,twitter,.@amazon delivery #drones show need to update ...


In [21]:
# Remove all values with an "audience confidence," "bias
# confidence," or "message confidence" score below 1.

df = df[(df['audience:confidence'] >= 1) & (df['bias:confidence'] >= 1) & (df['message:confidence'] >= 1)]

In [22]:
# Remove extra columns.

df = df[['_unit_id', '_trusted_judgments', 'audience',
         'bias', 'message', 'label', 'source', 'text']]

In [23]:
# Relabel columns.

df.columns = ['unit_id', 'trusted_judgments', 'audience_feature',
              'bias_feature', 'message_feature', 'label_feature',
              'source_feature', 'text_feature']

In [24]:
# Drop NAs.
df.dropna(inplace=True)


In [25]:
# Reset index.

df.reset_index(drop=True, inplace=True)

We have social media data! This includes almost 5,000 messages on either Twitter or Facebook from various politicians. We can use the features we generated to predict things like whether the source is Twitter or Facebook, whether the bias is neutral or partisan, and so on.

In [29]:
# How many rows and columns do we have?
df.shape


(4888, 8)

In [27]:
# View first five rows.

df.head()

,unit_id,trusted_judgments,audience_feature,bias_feature,message_feature,label_feature,source_feature,text_feature
0,766192484,1,national,partisan,policy,From: Trey Radel (Representative from Florida),twitter,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,766192485,1,national,partisan,attack,From: Mitch McConnell (Senator from Kentucky),twitter,VIDEO - #Obamacare: Full of Higher Costs and ...
2,766192486,1,national,neutral,support,From: Kurt Schrader (Representative from Oregon),twitter,Please join me today in remembering our fallen...
3,766192487,1,national,neutral,policy,From: Michael Crapo (Senator from Idaho),twitter,RT @SenatorLeahy: 1st step toward Senate debat...
4,766192488,1,national,partisan,policy,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...


You may note that there are some extra symbols in the data. This is a common problem in natural language processing, especially when dealing with social media (think emoji, hashtags, links, etc.), but we're going to ignore that for now.

### Let's use Naive Bayes to predict whether a social media post was featured on Facebook or Twitter.

In [30]:
df['source_feature'].value_counts(normalize=True)

facebook    0.510843
twitter     0.489157
Name: source_feature, dtype: float64

#### 1. Engineer a feature to turn `source_feature` into a 1/0 column, where 1 indicates `Twitter`.

In [31]:
df['twitter'] = df['source_feature'].apply(lambda x: 1 if x == 'twitter' else 0)

In [32]:
df['twitter'].value_counts(normalize=True)

0    0.510843
1    0.489157
Name: twitter, dtype: float64

#### NOTE: Since we are solving a classification problem, what potential issue should I check for here?

#### 2. Split our data into `X` and `y`.

In [33]:
X = df[['text_feature']]
y = df['twitter']

In [34]:
X.head()

,text_feature
0,RT @nowthisnews: Rep. Trey Radel (R- #FL) slam...
1,VIDEO - #Obamacare: Full of Higher Costs and ...
2,Please join me today in remembering our fallen...
3,RT @SenatorLeahy: 1st step toward Senate debat...
4,.@amazon delivery #drones show need to update ...


#### 3. Split our data into training and testing sets.

In [47]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    stratify=y)

#### 4. Turn our text into features. [Documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html).

In [48]:
# Import CountVectorizer.
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate our CountVectorizer.
cvec = CountVectorizer(
    max_features= 500,
)

In [49]:
# Fit our CountVectorizer on the training data and transform training data.
X_train_cvec = pd.DataFrame(cvec.fit_transform(X_train['text_feature']).todense(),
                          columns = cvec.get_feature_names())

In [50]:
# Transform our testing data with the already-fit CountVectorizer.
X_test_cvec = pd.DataFrame(cvec.transform(X_test['text_feature']).todense(),
                         columns = cvec.get_feature_names())

In [51]:
# feature analysis

# logistic regresion, we had coefficients 
# we used get feature names to observerve wheter one coefficient 
# on the given word was big or small 

# NB we had the relative feature_log_probs 
# we used get_feature_names to observe whether the relative 
# feature_log_prob for the given word was big or small 



In [52]:
X_train_compressed = cvec.fit_transform(X_train)
X_test_compressed  = cvec.transform(X_test)

#### 4. Fit a Naive Bayes model!

<details><summary> Which Naive Bayes model should we pick, and why? </summary>
    
- The columns of X are all integer counts, so MultinomialNB is the best choice here.
- BernoulliNB is best when we have 0/1 counts in all columns of X. (a.k.a. dummy variables)
- GaussianNB is best when the columns of X are Normally distributed. (Practically, though, it gets used whenever BernoulliNB and MultinomialNB are inappropriate.)
</details>

In [53]:
# Import our model!

from sklearn.naive_bayes import MultinomialNB

In [54]:
# Instantiate our model!

nb = MultinomialNB()

Remember earlier that I said we had the opportunity to set priors. We could do so here if we wanted, but we'll stick with the default and allow `sklearn` to estimate priors from the training data directly.

In [55]:
# Fit our model!

model = nb.fit(X_train_compressed, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [1, 3666]

In [56]:
# Generate our predictions!

predictions = model.predict(X_test_cvec)

NameError: name 'model' is not defined

<details><summary> How might we evaluate our model's performance? </summary>

- Accuracy = (TP + TN) / (TP + TN + FP + FN)
- Sensitivity = TP / (TP + FN)
- Specificity = TN / (TN + FP)
- Precision = TP / (TP + FP)
- AUC ROC
</details>

<details><summary> If we have to select only one, which one should we choose? </summary>

- It depends on how exactly you define "positive" and "negative." In this case, it probably doesn't really matter - incorrectly mistaking a tweet for a Facebook post doesn't seem much better or worse than incorrectly mistaking a Facebook post for a tweet. 
- Because I believe false positives and false negatives are equally as bad, I'd probably use accuracy.
</details>

In [ ]:
# Score our model on the training set.



In [ ]:
# Score our model on the testing set.



<details><summary> What should we do in this case? </summary>

- Our model appears *slightly* overfit. We could:
    - try to collect more data, 
    - try using fewer features by setting `max_features` to a smaller number when instantiating our CountVectorizer,
    - try TF-IDF Vectorizer,
    - try a non-default prior **if you have subject-matter expertise**.
- Rather than regularizing, [online answers](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) suggest using a different model entirely.
- Our training performance and testing performance are pretty close, though, so there may not be a lot of changes required.
</details>

In [ ]:
# Import the confusion matrix function.



In [ ]:
# Generate a confusion matrix.



In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()

In [ ]:
print("True Negatives: %s" % tn)
print("False Positives: %s" % fp)
print("False Negatives: %s" % fn)
print("True Positives: %s" % tp)

In [ ]:
# Feature analysis 
# --> for the given model, you have to know what the numbers are what they mean
# 
# logistic regression - look at the coefficients 
# multinomial NB - look at relative FLP and interpret
#  XYZ model -- ??


# confusion matrix 
# --> what kind of post does your model do best on 
# --> what kind of post does your model fail on 

# Make your own data to test your model 
#  --> ''To the moon" <-- test this 
# -- "Yesterday i went to NTUC and everyting so expensive!"

<details><summary> By default, what does a true negative mean here? </summary>

- True negatives are things we correctly predict to be negative.
- In this case, since Twitter = 1, a true negative means I correctly predict something is a Facebook post.
</details>

---

<details><summary> By default, what does a false positive mean here? </summary>

- False positives are things we falsely predict to be positive.
- In this case, since Twitter = 1, a false positive means I incorrectly preidct something is a tweet (when it's really a Facebook post).
</details>